<a href="https://colab.research.google.com/github/case485/FishingQuant/blob/main/LakeInfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip -q install python-weather
!pip -q install googlemaps
!pip -q install dash

In [40]:
import bs4
import requests
import requests, json 
import pandas as pd
import googlemaps
from datetime import datetime
import locale
import re
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pickle
from os import path

In [41]:
!cp drive/MyDrive/mySecrets.py .
import mySecrets

#API Keys
googleApiKey = mySecrets.lakeSecrets['googleApiKey']
home = mySecrets.lakeSecrets['home']
weatherApiKey = mySecrets.lakeSecrets['weatherApiKey']


In [42]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [43]:
def getLakeList(url):
  response = requests.get(url)
  df = pd.DataFrame(columns =['Lake Name'])
  baseLakeUrl = "https://tpwd.texas.gov/fishboat/fish/recreational/lakes/"
  soup = bs4.BeautifulSoup(response.content)
  lakeList = soup.find("div", {"class": "list2col"})
  lakeList = lakeList.find_all(href=True)
  print(lakeList)
  targetLakeList = []
  lakeRefName =[]
  for lake in lakeList: 
    lakeRefName.append(lake['href'])
    targetLakeList.append(baseLakeUrl + lake['href'])
  df["URL"] = targetLakeList
  df['Lake Name Orig'] =     lakeRefName
  df['Reference Name'] = df['Lake Name Orig'].str.replace('_','-')
  df["Lake Name"] = [s + " Lake" for s in lakeRefName]
  return(df)

In [44]:
def getRatings(lake):
  try:
    response = requests.get(lake)
    soup = bs4.BeautifulSoup(response.content)
    ratings = soup.find("table", {"id": "Ratings"})
    trList = ratings.find_all("tr")
    tdList = trList[1].find_all("td")
    largemouth = '<td><img alt="yes" class="nomargin" src="../images/icons/largemouth.gif"/></td>' 
    formalLakeName = soup.select('h1')[0].text.strip()
  except: 
    return("Not Found", "Not Found")
  try :
    tdList.index("yes")
  except:
    pass
  i=0
 
  for td in tdList:
    x = td.find("img")
    if(x):
      if i == 0:
        ranking = "Poor"
      elif i ==1:
        ranking = "Fair"
      elif i ==2:
        ranking = "Good"
      elif i == 3:
        ranking = "Excelent"
      else:
        ranking = "Not Found"
    i = i +1
  return(ranking,formalLakeName )

In [45]:
def getLakeArea(url):
  area =0
  try:
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content)
    area = soup.find(text=re.compile(".*Surface acres:.* |.*Surface area:.*"  ))
    area = area.split()[2]
  except:
    area = 0
  if type(area) is str:
    area = locale.atof(area)
    return(area)
  return(area)

In [46]:
def getWaterLevel(lake):
  lake = lake.replace("_", "-")
  url = "https://waterdatafortexas.org/reservoirs/individual/" + lake
  response = requests.get(url)
  soup = bs4.BeautifulSoup(response.content)
  waterLevel = soup.find("small")
  if(type(waterLevel) is bs4.element.Tag ):
    waterLevel = waterLevel.get_text()
    waterLevel = waterLevel.split(" ")
    waterLevel = waterLevel[1]
  return(waterLevel)

In [47]:
def getWaterTempature(lake):
  url = "https://www.lakemonster.com/lake/TX/" + lake 
  response = requests.get(url)
  soup = bs4.BeautifulSoup(response.content)
  waterTemp = soup.find(id = "water_text")
  if(type(waterTemp) is bs4.element.Tag ):
    waterTemp = waterTemp.get_text()
    waterTemp = waterTemp.split(" ")
    waterTemp = waterTemp[0]
  return(waterTemp)

In [48]:
def getCityNamefromLake(lake):
  lake = lake + ", Tx"
  lake = lake.replace("/","")
  lake = lake.replace("-"," ")
  locationDict = {"zipCode" : None,
                  "lat" : None,
                  "lng" : None, 
                  "lakeName" : None
                  }
  gmaps = googlemaps.Client(key=googleApiKey)
  try: 
    geocode_result = gmaps.geocode(lake)
    try:
      locationDict["zipCode"] = geocode_result[0]["address_components"][4]["long_name"]
    except: 
      locationDict["zipCode"] = "11111"
    lat = geocode_result[0]["geometry"]["bounds"]["northeast"]["lat"]
    locationDict["lng"] = geocode_result[0]["geometry"]["bounds"]["northeast"]["lng"]
    locationDict["lakeName"] = lake
    try:
      locationDict["Location"] = geocode_result[0]["formatted_address"]
    except:
      locationDict["Location"] = "N/A"
  except: 
    print("Error")
    return("N/A", "N/A", "N/A", "N/A")
  return(locationDict["Location"], locationDict["zipCode"], lat, locationDict["lng"])

In [49]:
def getDistanceFromHome(location):
  distance = "None"
  gmaps = googlemaps.Client(key=googleApiKey)
  location = location + " Tx"
  try:
    distanceTemp = gmaps.distance_matrix(home, location,units = "imperial")
    distance = distanceTemp["rows"][0]['elements'][0]["distance"]["text"]
    distance = distance.split()[0]
  except:
    distance = "0"
  return(float(distance))

In [50]:
def getShareALunkerLakes():
  url = "https://texassharelunker.com/mobifish/services/sharelunker/query/sl_query.aspx?op=downloadCSV"
  df = pd.read_csv(url)
  df.reset_index(inplace=True)
  df = df.rename(columns=df.iloc[0]).drop(df.index[0])
  lakesSeries = df['lake_name'].value_counts()
  return(lakesSeries)

In [51]:
def addLunkers(lake, lunkerLakesSeries):
  lake = lake.replace("/", "")
  lake = lake.replace("_", " ")
  lake = lake.replace("-", " ")
  lake = lake.title()
  substring = lake.strip()
  strings_with_substring = [string for string in lunkerLakesSeries.keys().to_list() if substring in string]
  try:
    lunkers = lunkerLakesSeries[strings_with_substring][0]
  except:
    lunkers = 0
  return(lunkers)

In [52]:
def getWeather(city_name="Waco"):
  print("****************************************************")
  print(f"Orig City : {city_name}")
  city_name = city_name.replace('Lake','')
  if(city_name == "N/A"):
    return(0, 0)
  current_temperature = 0 
  current_pressure= 0
  try:
    print(f"Adj City :{city_name}")
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + weatherApiKey + "&q=" + city_name 
    print(complete_url)
  except:
    print("Error on City Name Resolution")
    return(0, 0)
  try:
    x = requests.get(complete_url) 
    x = x.json()
    print(x["main"])
    y = x["main"] 
    current_temperature = int(1.8*(int(y["temp"]) - 273) + 32)
    print(type(current_temperature))
    fTemp = 1.8*(current_temperature - 273) + 32
    current_pressure = y["pressure"] 
    current_humidiy = y["humidity"] 
    z = x["weather"] 
    weather_description = z[0]["description"] 
    print(" Temperature (F) = " +
                    str(current_temperature) + 
          "\n atmospheric pressure (in hPa unit) = " +
                    str(current_pressure) +
          "\n humidity (in percentage) = " +
                    str(current_humidiy) +
          "\n description = " +
                    str(weather_description)) 
  except: 
    print("Error")
  return(current_temperature, current_pressure)

In [53]:
def getRegionalList():
  filePath = f"/content/drive/MyDrive/Colab Notebooks/lakeData.pkl"
  if (path.exists(filePath)):
    print(f"File already exists, reading in pickle")
    dfLake = pd.read_pickle(filePath)
  else: 
    dfLake = pd.DataFrame(columns =['Lake Name'])
    urlList = ["https://tpwd.texas.gov/fishboat/fish/recreational/lakes/inplains.phtml", "https://tpwd.texas.gov/fishboat/fish/recreational/lakes/ineast.phtml", "https://tpwd.texas.gov/fishboat/fish/recreational/lakes/inhillco.phtml","https://tpwd.texas.gov/fishboat/fish/recreational/lakes/inpanhd.phtml"]
    print(f"URLList: {urlList}")
    for url in urlList:
      dfLake = pd.concat([dfLake, getLakeList(url)])
    dfLake.to_pickle(filePath)
  return(dfLake)

In [54]:
def getBassRecord(url):
  baseUrl = "https://tpwd.texas.gov/fishboat/fish/action/waterecords.php?"
  try: 
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content)
    for link in soup.findAll('a', href=True, text="Lake Records"):
      wbCode = link['href'].split("?")[1]
    recordUrl = baseUrl + wbCode
    df_list = pd.read_html(recordUrl)
    rodRecords = df_list[0]
    bassRecord = rodRecords.loc[rodRecords['Species'] == 'Bass, Largemouth']
    bassRecord = bassRecord['Weight'].values[0]
  except:
    bassRecord = 0
  return(float(bassRecord))

In [55]:
def buildDataframe(dfLake):
  filePath = f"/content/drive/MyDrive/Colab Notebooks/lakeInfo.pkl"
  if (path.exists(filePath)):
    dfLake = pd.read_pickle(filePath)
  else: 
    dfLake[['Ratings','Formal Lake Name']] = dfLake.apply(lambda row: pd.Series(getRatings(row["URL"])), axis=1)
    dfLake['Area'] = dfLake.apply(lambda row: getLakeArea(row["URL"]), axis = 1) 
    dfLake['Water Level'] = dfLake.apply(lambda row: getWaterLevel(row["Reference Name"]), axis = 1) 
    dfLake['Water Temp'] = dfLake.apply(lambda row: getWaterTempature(row["Reference Name"]), axis = 1) 
    dfLake[['Location', "Zip Code", "Lat", "Lng"]] = dfLake.apply(lambda row: pd.Series(getCityNamefromLake(row["Reference Name"])), axis = 1) 
    dfLake['Distance'] = dfLake.apply(lambda row: getDistanceFromHome(row["Location"]), axis = 1) 
    lunkerLakesSeries = getShareALunkerLakes()
    dfLake['Lunkers'] = dfLake.apply(lambda row: addLunkers(row["Reference Name"], lunkerLakesSeries), axis = 1) 
    dfLake["Lunkers per Acre"] = round( dfLake['Lunkers']/ dfLake["Area"] , 4)
    dfLake[["Air Temperature", "Atmospheric Pressure"]] = dfLake.apply(lambda row: pd.Series(getWeather(row["Location"])), axis = 1)
    dfLake["Bass Record"] =  dfLake.apply(lambda row: getBassRecord(row["URL"]), axis = 1) 
    dfLake.to_pickle(filePath)
  return(dfLake)

In [56]:
dfLake = getRegionalList()
# dfLake = dfLake.iloc[:3]

URLList: ['https://tpwd.texas.gov/fishboat/fish/recreational/lakes/inplains.phtml', 'https://tpwd.texas.gov/fishboat/fish/recreational/lakes/ineast.phtml', 'https://tpwd.texas.gov/fishboat/fish/recreational/lakes/inhillco.phtml', 'https://tpwd.texas.gov/fishboat/fish/recreational/lakes/inpanhd.phtml']
[<a href="alvarado">Alvarado Park</a>, <a href="amon_carter">Amon G. Carter</a>, <a href="aquilla">Aquilla</a>, <a href="arlington">Arlington</a>, <a href="athens">Athens</a>, <a href="bachman">Bachman</a>, <a href="bardwell">Bardwell</a>, <a href="bastrop">Bastrop</a>, <a href="belton">Belton</a>, <a href="benbrook">Benbrook</a>, <a href="big_creek">Big Creek</a>, <a href="bonham">Bonham City</a>, <a href="bonhamsp">Bonham State Park</a>, <a href="brazos">Brazos</a>, <a href="bridgeport">Bridgeport</a>, <a href="bryan">Bryan</a>, <a href="cedar_creek">Cedar Creek</a>, <a href="cleburnesp">Cleburne State Park</a>, <a href="coffee_mill">Coffee Mill</a>, <a href="cooper">Cooper</a>, <a href

In [57]:
dfLake = buildDataframe(dfLake)

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
****************************************************
Orig City : Alvarado, TX 76009, USA
Adj City :Alvarado, TX 76009, USA
http://api.openweathermap.org/data/2.5/weather?appid=9cb2ed20a1779b1e65bb507659fccd16&q=Alvarado, TX 76009, USA
{'temp': 298.96, 'feels_like': 299.55, 'temp_min': 298.96, 'temp_max': 298.96, 'pressure': 1013, 'humidity': 75, 'sea_level': 1013, 'grnd_level': 967}
<class 'int'>
 Temperature (F) = 77
 atmospheric pressure (in hPa unit) = 1013
 humidity (in percentage) = 75
 description = few clouds
****************************************************
Orig City : N/A
****************************************************
Orig City : Aquilla, TX 76622, USA
Adj City :Aquilla, TX 76622, USA
http://api.openweathermap.org/data/2.5/weather?appid=9cb2ed20a1779b1e65bb507659fccd16&q=Aquilla, TX 76622, U

In [58]:
dfLake

,Lake Name,URL,Lake Name Orig,Reference Name,Ratings,Formal Lake Name,Area,Water Level,Water Temp,Location,Zip Code,Lat,Lng,Distance,Lunkers,Lunkers per Acre,Air Temperature,Atmospheric Pressure,Bass Record
0,alvarado Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/alvarado,alvarado,alvarado,Good,Alvarado Park Lake,437.0,None,39°,"Alvarado, TX 76009, USA",76009,32.4353,-97.188,21.0,4,0.0092,77,1013,12.77
1,amon_carter Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/amon_carter,amon_carter,amon-carter,Excelent,Lake Amon G. Carter,1848.0,None,41°,N/A,N/A,N/A,N/A,60.9,0,0.0000,0,0,14.44
2,aquilla Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/aquilla,aquilla,aquilla,Fair,Aquilla Lake,3020.0,100.0%,66°,"Aquilla, TX 76622, USA",76622,31.8612,-97.2149,48.4,1,0.0003,53,1028,12.81
3,arlington Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/arlington,arlington,arlington,Excelent,Lake Arlington,1939.0,100.0%,71°,"Arlington, TX, USA",11111,32.8171,-97.037,34.5,10,0.0052,69,1016,13.38
4,athens Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/athens,athens,athens,Excelent,Lake Athens,1799.0,100.0%,57°,"Athens, TX, USA",11111,32.2353,-95.7195,78.3,66,0.0367,69,1015,14.19
5,bachman Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/bachman,bachman,bachman,Good,Bachman Lake,132.0,None,None,"Bachman Lake, Dallas, TX, USA",United States,32.8602,-96.8497,43.8,0,0.0000,0,0,7.80
6,bardwell Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/bardwell,bardwell,bardwell,Fair,Lake Bardwell,3138.0,100.0%,69°,"Bardwell, TX, USA",11111,32.2727,-96.6901,22.3,0,0.0000,51,992,10.44
7,bastrop Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/bastrop,bastrop,bastrop,Excelent,Lake Bastrop,906.0,None,69°,"Bastrop, TX 78602, USA",78602,30.1377,-97.2776,172.0,0,0.0000,71,1022,9.98
8,belton Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/belton,belton,belton,Good,Belton Lake,12385.0,100.0%,65°,"Belton, TX, USA",11111,31.1189,-97.426,106.0,3,0.0002,69,1015,13.97
9,benbrook Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/benbrook,benbrook,benbrook,Fair,Benbrook Lake,3635.0,100.0%,65°,"Benbrook, TX, USA",11111,32.7238,-97.4118,46.2,21,0.0058,69,1017,13.66


In [59]:
dfLake.sort_values(by="Lunkers", ascending=False)

,Lake Name,URL,Lake Name Orig,Reference Name,Ratings,Formal Lake Name,Area,Water Level,Water Temp,Location,Zip Code,Lat,Lng,Distance,Lunkers,Lunkers per Acre,Air Temperature,Atmospheric Pressure,Bass Record
5,fork Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/fork,fork,fork,Excelent,Lake Fork,27264.0,100.0%,69°,"Lake Fork, Texas, USA",11111,32.9801,-95.478,113.0,450,0.0165,0,0,18.18
4,conroe Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/conroe,conroe,conroe,Excelent,Lake Conroe,20118.0,100.0%,71°,"Conroe, TX, USA",11111,30.4199,-95.3904,188.0,121,0.0060,69,1016,15.93
24,sam_rayburn Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/sam_rayburn,sam_rayburn,sam-rayburn,Excelent,Sam Rayburn Reservoir,114500.0,100.0%,70°,N/A,N/A,N/A,N/A,60.9,90,0.0008,0,0,16.80
4,athens Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/athens,athens,athens,Excelent,Lake Athens,1799.0,100.0%,57°,"Athens, TX, USA",11111,32.2353,-95.7195,78.3,66,0.0367,69,1015,14.19
0,austin/ Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/austin/,austin/,austin/,Excelent,Lake Austin,1599.0,95.4%,None,"Austin, TX, USA",11111,30.5169,-97.5684,165.0,46,0.0288,68,1016,16.03
1,alan_henry Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/alan_henry,alan_henry,alan-henry,Excelent,Alan Henry Reservoir,2880.0,86.3%,55°,"Lake Alan Henry, Texas, USA",11111,33.0646,-101.029,307.0,40,0.0139,0,0,15.00
6,gilmer Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/gilmer,gilmer,gilmer,Excelent,Lake Gilmer,1010.0,None,~°,"Gilmer, TX, USA",11111,32.7626,-94.9205,151.0,32,0.0317,69,1019,14.00
19,nacogdoches Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/nacogdoches,nacogdoches,nacogdoches,Excelent,Nacogdoches Lake,2212.0,100.0%,64°,"Nacogdoches, TX, USA",11111,31.6734,-94.5935,166.0,29,0.0131,69,1018,15.34
11,lady_bird/ Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/lady_bird/,lady_bird/,lady-bird/,Excelent,Lady Bird (formerly Town Lake),468.0,None,None,N/A,N/A,N/A,N/A,60.9,25,0.0534,0,0,13.50
4,canyon/ Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/canyon/,canyon/,canyon/,Good,Canyon Lake,8308.0,89.6%,None,"Canyon, TX 79015, USA",79015,35.0286,-101.875,388.0,25,0.0030,59,1016,11.69


In [60]:
dfLake.loc[dfLake['Ratings'] == "Excelent"]

,Lake Name,URL,Lake Name Orig,Reference Name,Ratings,Formal Lake Name,Area,Water Level,Water Temp,Location,Zip Code,Lat,Lng,Distance,Lunkers,Lunkers per Acre,Air Temperature,Atmospheric Pressure,Bass Record
1,amon_carter Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/amon_carter,amon_carter,amon-carter,Excelent,Lake Amon G. Carter,1848.0,None,41°,N/A,N/A,N/A,N/A,60.9,0,0.0000,0,0,14.44
3,arlington Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/arlington,arlington,arlington,Excelent,Lake Arlington,1939.0,100.0%,71°,"Arlington, TX, USA",11111,32.8171,-97.037,34.5,10,0.0052,69,1016,13.38
4,athens Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/athens,athens,athens,Excelent,Lake Athens,1799.0,100.0%,57°,"Athens, TX, USA",11111,32.2353,-95.7195,78.3,66,0.0367,69,1015,14.19
7,bastrop Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/bastrop,bastrop,bastrop,Excelent,Lake Bastrop,906.0,None,69°,"Bastrop, TX 78602, USA",78602,30.1377,-97.2776,172.0,0,0.0000,71,1022,9.98
22,davy_crockett Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/davy_crockett,davy_crockett,davy-crockett,Excelent,Davy Crockett Lake,355.0,None,44°,N/A,N/A,N/A,N/A,60.9,2,0.0056,0,0,12.59
25,fayette Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/fayette,fayette,fayette,Excelent,Fayette County Reservoir,2400.0,None,62°,"Fayette County, TX, USA",11111,30.1644,-96.5698,191.0,2,0.0008,0,0,12.25
27,gibbons_creek Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/gibbons_creek,gibbons_creek,gibbons-creek,Excelent,Gibbons Creek Reservoir,2770.0,82.9%,69°,"Gibbons Creek Reservoir, Texas 77830, USA",77830,30.6561,-96.0276,161.0,5,0.0018,0,0,16.13
30,grapevine Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/grapevine,grapevine,grapevine,Excelent,Grapevine Lake,6684.0,100.0%,69°,"Grapevine, TX, USA",11111,32.9936,-97.0158,47.2,0,0.0000,46,1015,12.09
45,nocona Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/nocona,nocona,nocona,Excelent,Lake Nocona,1362.0,100.0%,47°,"Nocona, TX 76255, USA",76255,33.8026,-97.7073,126.0,6,0.0044,68,1018,13.40
50,purtis_creek Lake,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/purtis_creek,purtis_creek,purtis-creek,Excelent,Purtis Creek State Park Lake,349.0,None,54°,"Purtis Creek, Texas 75124, USA",75124,32.3793,-95.9674,64.7,19,0.0544,0,0,13.73


In [61]:
df = dfLake
x = (pd.interval_range(start=dfLake['Lunkers per Acre'].min(), end= dfLake['Lunkers per Acre'].max(), periods=4))
x = x.to_tuples().to_list()
limits = [(0,2),(3,10),(11,20),(21,50),(50,3000)]
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
cities = []
scale = 5000
fig = go.Figure()
for i in range(len(limits)):
    lim = limits[i]
    df_sub = dfLake[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = dfLake['Lng'],
        lat = dfLake['Lat'],
        text = dfLake['Location'],
        marker = dict(
            size = dfLake['Lunkers per Acre'] *10000,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))
fig.update_layout(
        title_text = 'Texas Lakes by Lunker Density',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()

In [62]:
dfLake = dfLake[["Formal Lake Name", "Ratings", "Bass Record", "Lunkers per Acre" , "Lunkers", "Distance","URL" ]].sort_values(by="Distance")

In [63]:
dfLake[~dfLake.Ratings.isin(["Fair", "Poor"])].sort_values(by="Bass Record", ascending=False)

,Formal Lake Name,Ratings,Bass Record,Lunkers per Acre,Lunkers,Distance,URL
5,Lake Fork,Excelent,18.18,0.0165,450,113.0,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/fork
21,Lake Pinkston,Excelent,16.90,0.0115,6,193.0,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/pinkston
24,Sam Rayburn Reservoir,Excelent,16.80,0.0008,90,60.9,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/sam_rayburn
39,Mill Creek Reservoir,Good,16.77,0.0633,15,60.9,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/mill_creek
3,Caddo Lake,Excelent,16.17,0.0005,13,124.0,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/caddo
27,Gibbons Creek Reservoir,Excelent,16.13,0.0018,5,161.0,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/gibbons_creek
0,Lake Austin,Excelent,16.03,0.0288,46,165.0,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/austin/
39,Possum Kingdom Reservoir,Good,16.02,0.0010,15,147.0,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/possum_kingdom
4,Lake Conroe,Excelent,15.93,0.0060,121,188.0,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/conroe
19,Nacogdoches Lake,Excelent,15.34,0.0131,29,166.0,https://tpwd.texas.gov/fishboat/fish/recreational/lakes/nacogdoches
